# Preamble

This notebook collects analysis and visualization of all data related to nitrate inventories and other mixed-layer properties as compiled in [the previous notebook]().

All figures exported from this notebook are prefixed with `FIGURE_FN-COMP_`.

In [ ]:
%load_ext autoreload
%autoreload 2
%run imports.py

In [ ]:
pd.read_feather('../data/no3-compilation/da')

In [ ]:
def load_data():
    # df_per_station, df = load_data()
    df_per_station = pd.read_feather('../data/no3-compilation/database-per-stn.feather')
    df = pd.read_feather('../data/no3-compilation/database.feather')
    return df_per_station, df

In [ ]:
# Define dimensions
dims = dict(
    reg_name=hv.Dimension('reg_name', label='Region'),
    ntr0=hv.Dimension('ntr0', label='Sfc. NO₃', unit='µM', range=(0,13)),
    doy=hv.Dimension('doy', label='Day of the year', range=(0,360)),
    lon=hv.Dimension('lon', label='Longitude', unit='°E'),
    lat=hv.Dimension('lat', label='Latitude', unit='°N'),
    delta_sigth=hv.Dimension('delta_sigth', label='Δσ-θ', unit='kg m⁻³'),
    strat=hv.Dimension('strat', label='N²', unit='s⁻²'),
    logstrat=hv.Dimension('logstrat', label='log10 N²', unit='s⁻²'),
    no3_sfc_winter = hv.Dimension('no3_sfc_winter', label='Pre-bloom sfc. NO₃ ', unit='µM', range=(-.5,13))
)

# Maps of key quantities

In [ ]:
df, dfp, _ = load_data()

In [ ]:
def logmean(x):
    return np.nanmedian(np.log10(x))

options = (
    opts.HexTiles(),
    opts.HexTiles('ntr0', cmap=cmocean.cm.matter_r),
    opts.HexTiles('delta_sigth', aggregator=logmean),
    opts.HexTiles('no3sig_nc', color_levels=8),
    opts.HexTiles('no3sig_mld', color_levels=8),
    opts.HexTiles('deltanc', symmetric=True, cmap='PuOr'),
    opts.Polygons(line_color='k', fill_color=None, line_width=3),
    opts.Shape(line_width=1, color='w'),
)

griddict = {
    (season, var):
    gv.HexTiles(df.loc[df.season==season].dropna(subset=[var]), ['lon', 'lat'], var, label=var) 
    for season in ['winter', 'summer']
    for var in ['nc', 'mld', 'ntr0', 
                'delta_sigth']

}


gs = hv.GridSpace(griddict, kdims=['season', 'variable'])
gs = gs.redim.range(deltanc=(-50, 50), mld=(0,100), nc=(0.1,100), nc0=(0.1,60))#, delta_sigth=(.05,3))
l = gs * land

## html 

## mpl output

Make custom diverging color map:

In [ ]:
# cm = colorcet.b_diverging_linear_bjy_30_90_c45[::-1]
cm = colorcet.b_diverging_bwr_40_95_c42[::-1]
cm = cm[:127:4] + cm[127:]

In [ ]:
obj = l.opts(
    opts_map_mpl + [
        opts.HexTiles(clim=(0,5), backend='matplotlib'),
    ]
)

In [ ]:
fig = mplrender_map(obj['summer', 'ntr0'], fname='../nb_fig/FIGURE_NO3-COMP_map_no3_summer')

fig

In [ ]:
fig = mplrender_map(obj['winter', 'ntr0'], fname='../nb_fig/FIGURE_NO3-COMP_map_no3_winter')

fig

# Seasonality

In [ ]:
df, dfp = load_data()

## Seasonality by regions

In [ ]:
loc = gv.Points(df, ['lon', 'lat'], [])
ntr = hv.Scatter(df, 'doy', ['ntr0'])

DataLink(loc,ntr)

l = loc*land*gv.feature.rivers + ntr 
l = l.redim(**dims).cols(1)

options = (
    opts.Scatter(
        tools=['lasso_select', 'box_select'], active_tools=['lasso_select'], 
        color='lightblue', size=5, selection_color='orange',
        height=300, width=600
    ),
    opts.Points(
        tools=['lasso_select'], active_tools=['lasso_select'],
        projection=ccrs.NorthPolarStereo(),
        color='lightblue', size=3, selection_color='orange',
        height=400, aspect=1,
    ),
    opts.Feature(
        fill_color='wheat', line_color='black',
        scale='50m'
    ),
    opts.Feature('Rivers', line_color='blue', scale='110m'),
)
l = l.opts(*options)

hv.save(l, '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle.html')
l

## Make static matplotlib plot for paper 

In [ ]:
def annual_cycle(v):
    """Plots seasonal cycle of variable v"""
    sc = hv.Scatter(df, kdims=['doy'], 
                      vdims=[v, 
                             'nc', 'lat', 'lon', 'mld', 'ntr0', 'nc0', 'reg_name'], 
                      label=v
                     ).groupby('reg_name')
    err = hvu.bin_average(sc, bins=np.arange(0,361,30))
    return sc *  err * err.map(hv.Curve, hv.Dataset) * err.map(hv.Scatter, hv.Dataset)

l = annual_cycle('ntr0').layout().cols(3)
l = l.opts(*opts_timeseries_mpl + [
    opts.Scatter(title_format='asd{group}',
        invert_yaxis=False, backend='matplotlib', 
        xlabel='', #ylabel='',
    ),
    opts.Layout()
])
l = l.redim(**dims)

# hv.save(l, '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle.png')
with plt.style.context('styles/sparse.mplstyle'):
    fig = mplrender(l)
    fig.subplots_adjust(wspace=.25, hspace=.08, left=0.2)
    fig.set_size_inches((12,8))
    
fname = '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle'
fig.savefig(fname+'.png')
fig.savefig(fname+'.pdf')
fig

# Laptev Nitrate profiles

In [ ]:
df_single, df = load_data()

In [ ]:
(
    df.loc[df.lat.between(76, 77) & df.lon.between(124, 128)]
    .hvplot('nitrate', 'depth', by='station')
    .opts(invert_yaxis=True)
)

In [ ]:
(
    df.loc[df.lat.between(76, 77) & df.lon.between(124, 128)]
    .hvplot('sigth', 'depth', by='station')
    .opts(invert_yaxis=True)
)

Gradient:

In [ ]:
3/20

In [ ]:
(27-23)/20

# [do later] Interannual trends

obs:

- NC deepening in S Beaufort
- 

to do:
make 2dim hextiles aggregator, then plot hextiles of trends and significance

In [ ]:
df_per_station, df = load_data()

df = df.melt(['year', 'season', 'reg_name'], ['ntr0', 'nc', 'nc0']).dropna(subset=['season'])
df = df.loc[~df.reg_name.isnull()]

In [ ]:
df.reg_name.unique()

In [ ]:
m = df.hvplot.scatter('year', 'value', groupby=['reg_name', 'season', 'variable'])

from utils.holoviews.element import Regression

l=m*m.map(Regression, 'Scatter')

In [ ]:
%%opts Scatter [ frame_width=600 frame_height=400 show_grid=True invert_yaxis=True, legend_position='top']
l.overlay('season')#.layout('reg_name').cols(1)

## [unused] Scatter plots: Relations

deltanc vs surface NO3

NC vs MLD...